# Assignment 4: Contrastive Learning

## Meta Instructions
1. Environment: If you use Google Colab, the default environment is OK. For your own environment, install pytorch >= 1.7.0 and torchvision >= 0.8.0 to avoid any issues caused by package versions.

2. Finish coding tasks according to instructions in this file. **Only** change the code/text in the required area, which is indicated by "Your code/answer starts here" and "Your code/answer ends here". Do not modify anywhere else including comments. 1 GPU is sufficient for this task.

2. Submission: submit a zip file named "StuID.zip" (e.g., "A0123456J.zip") to Canvas. Note that it is **NOT** NUSNET ID. The zip file should **only** include "main.ipynb" with your implemented code. **Keep the intermediate results in this notebook. We won't run it for you.** The submissison deadline is **23:59 Mar 10**.


Please **strictly** follow the above instructions, otherwise a grade deduction will be conducted.

For any question, please do one of the following actions with priority:
1. Search for similar questions on Slack
2. Propose a new question on Slack
3. For non-public questions, e-mail to Xiangyu Peng (e0792500@u.nus.edu) with the subject starting with "CS5260 2023 Spring"

## Get Started

In this assignment, we will learn how to learn from unlabeled data using contrastive learning. The idea of contrastive learning is to pull positive pairs closer while pushing apart negative samples in the latent space. Specifically, we are going to implement a classical method called [**SimCLR**](https://arxiv.org/abs/2002.05709). The algorithm of **SimCLR** is shown below.
<img src="./resources/alg.jpg" alt="Alternative text" width="500"/>

## Load the CIFAR-10 dataset

In this task, we use CIFAR-10 for training. CIFAR-10 is a small dataset with category labels. Since our task is self-supervised learning, we will only use the images but not the labels. Let's first prepare the dataset.

In [1]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision


class Args:
    root = './data'
    epochs = 10
    batch_size = 64
    lr = 0.1
    latent_dim = 256
    temperature = 0.1
    topk = 199
    log_interval = 100
    
args = Args()

In [2]:
from torchvision.datasets import CIFAR10
from torchvision import transforms

mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.2010)
class MultiViewTransform:
    """Create multiple views of the same image"""
    def __init__(self, num_views=2):

        transform = transforms.Compose([
            transforms.RandomResizedCrop(size=32, scale=(0.2, 1.0)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomApply([
                transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)
            ], p=0.8),
            transforms.RandomGrayscale(p=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])

        transform = [transform for _ in range(num_views)]
        self.transforms = transform

    def __call__(self, x):
        views = [t(x) for t in self.transforms]
        return views

transform_train = MultiViewTransform()
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

train_set = CIFAR10(root=args.root, train=True, transform=transform_train, download=True)
eval_train_set = CIFAR10(root=args.root, train=True, transform=transform_test)
test_set = CIFAR10(root=args.root, train=False, transform=transform_test)


train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=args.batch_size,
    num_workers=10,
    shuffle=True,
    drop_last=True,
)
eval_train_loader = torch.utils.data.DataLoader(
    eval_train_set,
    batch_size=args.batch_size,
    num_workers=10,
    shuffle=False,
    drop_last=False,
)
test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=args.batch_size,
    num_workers=10,
    shuffle=False,
    drop_last=False,
)

Files already downloaded and verified


## Prepare the model

We use a small model ResNet-18 for fast training. We remove the maxpool as CIFAR-10 images are already very small. The fully connected layer is replaced by a two-layer MLP as in **SimCLR**.

In [3]:
model = torchvision.models.resnet18(pretrained=False, num_classes=args.latent_dim)
model.maxpool = nn.Identity()
dim_mlp = model.fc.weight.shape[1]
model.fc = nn.Sequential(nn.Linear(dim_mlp, dim_mlp), nn.ReLU(), model.fc)
model.cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, weight_decay=1e-4)

## Contrastive Loss

Next, we are going to implement the contrastive loss in the algorithm shown above, which is the key to contrastive learning.

In [4]:
class ContrastiveLoss(nn.Module):
    def __init__(self, temperature):
        super(ContrastiveLoss, self).__init__()
        self.temperature = temperature
        self.ce = nn.CrossEntropyLoss()

    def forward(self, feat1, feat2):
        """
        Args:
            feat1, feat2: 
                features of two different views of the same batch.
                both are torch.Tensor of shape [batch_size, C].
            
        TODO: 
            Implement the contrastive loss defined in the algorithm above. 
            You can first have a look at the training process below to gain an idea of the input. 
        Hints:
            1. Do you find the format of contrastive loss is very similar to CrossEntropyLoss?  
                You can use self.ce in your implementation, but not a must.
            2. A concise implementation is within 10 lines (just reminding, not a must).
        Requirement:
            1. If you use self.ce, please follow the protocol of pytorch < 1.13 for nn.CrossEntropyLoss. See https://pytorch.org/docs/1.9.0/generated/torch.nn.CrossEntropyLoss.html?highlight=crossentropyloss#torch.nn.CrossEntropyLoss
        """
        loss = None
        ################################
        # Your code starts here
        ################################
        
        batch_size = feat1.shape[0]
        feats = torch.cat([feat1, feat2], dim=0)
        feats = F.normalize(feats, dim=1)
        logits = torch.matmul(feats, feats.T) / self.temperature
        
        mask = torch.eye(batch_size * 2, dtype=torch.bool).to(feat1.device)
        mask = ~mask
        logits = logits[mask].view(2 * batch_size, -1)
        
        targets = torch.arange(batch_size * 2, dtype=torch.long).to(feat1.device)
        targets = torch.cat([targets[batch_size:] - 1, targets[:batch_size]])
        
        loss = self.ce(logits, targets)
        ################################
        # Your code ends here
        ################################
        return loss

In [5]:
criterion = ContrastiveLoss(temperature=args.temperature)

## Train the model

We have already prepared the training code for you. You can start training once you are done with the implementation of contrastive loss. The training time on a single GPU should be no longer than 10 minutes.

In [6]:
def train(model, loader, criterion, args):
    model.train()
    num_iters = len(loader)
    print('==> Start training...')
    for epoch in range(1, args.epochs + 1):
        for idx, (images, _) in enumerate(loader, start=1):
            bsz = images[0].shape[0]
            images = torch.cat([images[0], images[1]], dim=0)
            images = images.cuda()

            # forward
            features = model(images)  # (2*bsz, C)
            f1, f2 = torch.split(features, [bsz, bsz], dim=0)
            loss = criterion(f1, f2)

            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if idx % args.log_interval == 0:
                print(f'Epoch [{epoch}/{args.epochs}][{idx}/{num_iters}] - '
                      f'loss: {loss.item():.2f}')
    print('==> Training finished.')

In [7]:
train(model, train_loader, criterion, args)

==> Start training...
Epoch [1/10][100/781] - loss: 4.04
Epoch [1/10][200/781] - loss: 4.11
Epoch [1/10][300/781] - loss: 3.93
Epoch [1/10][400/781] - loss: 3.35
Epoch [1/10][500/781] - loss: 3.55
Epoch [1/10][600/781] - loss: 3.38
Epoch [1/10][700/781] - loss: 3.68
Epoch [2/10][100/781] - loss: 3.11
Epoch [2/10][200/781] - loss: 2.94
Epoch [2/10][300/781] - loss: 3.40
Epoch [2/10][400/781] - loss: 2.94
Epoch [2/10][500/781] - loss: 2.68
Epoch [2/10][600/781] - loss: 2.33
Epoch [2/10][700/781] - loss: 2.27
Epoch [3/10][100/781] - loss: 2.32
Epoch [3/10][200/781] - loss: 2.21
Epoch [3/10][300/781] - loss: 2.54
Epoch [3/10][400/781] - loss: 2.33
Epoch [3/10][500/781] - loss: 2.89
Epoch [3/10][600/781] - loss: 1.94
Epoch [3/10][700/781] - loss: 2.32
Epoch [4/10][100/781] - loss: 2.14
Epoch [4/10][200/781] - loss: 2.39
Epoch [4/10][300/781] - loss: 1.95
Epoch [4/10][400/781] - loss: 2.25
Epoch [4/10][500/781] - loss: 2.17
Epoch [4/10][600/781] - loss: 2.48
Epoch [4/10][700/781] - loss: 1.9

## Test the model

We use the K-Nearest Neighbors (KNN) algorithm to test classification accuracy of the trained model. This requires your implementation. Specifically, we first need to build a feature bank of training samples for indexing. Then, we get the feature of a test sample, and find the topk neareast training samples in the feature bank. Last, the majority label of the topk nearest neighbours is assigned to the test sample as the predicted label.

In [8]:
def get_feature_bank(model, loader):
    with torch.no_grad():
        model.eval()
        feature_bank = []
        label_bank = []
        for images, labels in tqdm(loader):
            images = images.cuda()
            labels = labels.cuda()

            features = model(images)
            feature_bank.append(features)
            label_bank.append(labels)
        feature_bank = torch.cat(feature_bank, dim=0)  # [N, C]
        label_bank = torch.cat(label_bank, dim=0)  # [N]

    return feature_bank, label_bank

In [9]:
feature_bank, label_bank = get_feature_bank(model, eval_train_loader)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 782/782 [00:11<00:00, 67.99it/s]


In [10]:
def test(model, loader, feature_bank, label_bank):
    print('==> Testing accuracy on the test set...')
    model.eval()
    with torch.no_grad():
        gt_labels_all = []
        pred_labels_all = []
        test_bar = tqdm(loader)
        for images, labels in test_bar:
            bsz = images.shape[0]
            images = images.cuda()
            labels = labels.cuda()
            features = model(images)
            
            """
            TODO: 
                Implement the KNN algorithm for testing.  
            Hints:
                1. Use torch.gather() for indexing.
                1. Use torch.mode() to get the majority value.
                3. A concise implementation is within 6 lines (just reminding, not a must). 
            Requirement:
                1. Use cosine similarity as the distance metric for KNN.
                2. Use args.topk as the value of k.
            """
            pred_labels = torch.zeros_like(labels)
            
            ################################
            # Your code starts here
            ################################
            features = F.normalize(features, dim=1)
            feature_bank = F.normalize(feature_bank, dim=1)
            sim_matrix = torch.mm(features, feature_bank.T)  # [B, N]
            sim_indices = sim_matrix.topk(k=args.topk, dim=1)[1]  # [B, k]
            knn_labels = torch.gather(label_bank.expand(bsz, -1), dim=1, index=sim_indices)  # [B, k]
            pred_labels = torch.mode(knn_labels, dim=1)[0]
            ################################
            # Your code ends here
            ################################

            gt_labels_all.append(labels)
            pred_labels_all.append(pred_labels)

            cur_gt_labels = torch.cat(gt_labels_all)
            cur_pred_labels = torch.cat(pred_labels_all)
            acc = (cur_gt_labels == cur_pred_labels).float().mean() * 100
            test_bar.set_description(f'Test Acc: {acc.item():.2f}%')
        print(f'Final Test Acc: {acc.item():.2f}%')

In [11]:
test(model, test_loader, feature_bank, label_bank)

==> Testing accuracy on the test set...


Test Acc: 51.81%: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:02<00:00, 64.79it/s]

Final Test Acc: 51.81%


With correct implementation of contrastive loss and KNN, the test accuracy should be at least 49%.

## Visualization

Another intuitive way to judge the quality of features is to visualize the features. As features of neural networks have high dimensions, we need to reduce dimensions to 2D and then visualize them as 2D points. In this task, you are required to visualize high-dimensional features with the useful tool t-SNE. Scikit-learn has provided the [API](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html#:~:text=t%2DSNE%20%5B1%5D%20is,and%20the%20high%2Ddimensional%20data.). But you can also use other packages (or implement by your self :)). You don't need to visualize all samples, but only 3 classes with 100 samples per class.

In [12]:
def get_subset_feature_bank(feature_bank, label_bank, num_classes=3, samples_pc=100):
    feature_bank_subset = []
    label_bank_subset = []
    for idx in range(num_classes):
        inds_i = label_bank == num_classes
        feats_i = feature_bank[inds_i][:samples_pc]
        labels_i = label_bank[inds_i][:samples_pc]

        feature_bank_subset.append(feats_i)
        label_bank_subset.append(labels_i)
    feature_bank_subset = torch.cat(feature_bank_subset)
    label_bank_subset = torch.cat(label_bank_subset)
    return feature_bank_subset, label_bank_subset

In [13]:
feature_vis, label_vis = get_subset_feature_bank(feature_bank, label_bank)
print(feature_vis.shape)

torch.Size([300, 256])


In [14]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def plot_tsne(feats, labels):
    """
    Args:
        feats: torch.Tensor of shape [N, C]
        labels: torch.Tensor of shape [N]
    TODO: 
        Visualize features as 2D points using t-SNE.
    Hints:
        1. Use sklearn.manifold.TSNE to reduce feature dimensions (suggested)
        2. Use matplotlib.pyplot for visualization (suggested)
    """
    ################################
    # Your code starts here
    ################################
    
    tsne = TSNE(n_components=2)
    z = tsne.fit_transform(feats.cpu().numpy())
    plt.scatter(z[:,0],z[:,1],c=labels.cpu().numpy())
    plt.show()

    ################################
    # Your code ends here
    ################################

In [15]:
plot_tsne(feature_vis, label_vis)

An example of 10-class t-SNE looks like this. Your plot may not have such a good and clear clustering because we only train the model for 10 epochs. But the minimum requirement is different classes are marked in different colors.
<img src="./resources/tsne.png" alt="Alternative text" width="500"/>

One question: What do you think of the effect that a smaller temperature has on the t-SNE visualization? What about a larger temperature?

In [2]:
################################
# Your answer starts here
################################
"""
In a decent range, a smaller temperature tends to penalize the neighbour samples, which are probably 
of the same class as the anchor sample. In other words, a smaller temperature tends to push the neighbour samples 
harder, which leads to a more uniform distribution in the final t-SNE visualization. On the contrary, a larger
temperature is more tolerant to the neighbour samples, which results in more obvious clustering. Below are some
visualizations of temperatures varying from 0.05 to 5.0. Pay attention to the distribution of the same color,
i,e, samples from the same class.
"""

################################
# Your answer ends here
################################

t=0.05
<img src="./resources/t0.05.jpg" alt="Alternative text" width="400"/>
t=0.1
<img src="./resources/t0.1.jpg" alt="Alternative text" width="400"/>
t=0.5
<img src="./resources/t0.5.jpg" alt="Alternative text" width="400"/>
t=1.0
<img src="./resources/t1.0.jpg" alt="Alternative text" width="400"/>
t=5.0
<img src="./resources/t5.0.jpg" alt="Alternative text" width="400"/>